In [1]:
import mne
import os
import numpy as np

In [11]:
import pandas as pd
import numpy as np


data_path = "data/emotiv/EP1.01.txt"

df_emotiv = pd.read_csv(data_path, sep='\t', header=None)

df_emotiv.columns = ['id', 'event', 'device', 'channel', 'code', 'size', 'data']
df_emotiv['data'] = df_emotiv['data'].apply(lambda x: list(map(float, x.split(','))))
df_emotiv.head()

,id,event,device,channel,code,size,data
0,67635,67635,EP,AF3,6,260,"[4395.384615, 4382.564102, 4377.435897, 4387.1..."
1,67636,67635,EP,F7,6,260,"[4489.230769, 4475.384615, 4474.358974, 4486.6..."
2,67637,67635,EP,F3,6,260,"[4538.461538, 4528.717948, 4524.615384, 4526.1..."
3,67638,67635,EP,FC5,6,260,"[4207.692307, 4205.641025, 4200.51282, 4194.35..."
4,67639,67635,EP,T7,6,260,"[4497.948717, 4498.461538, 4494.871794, 4497.9..."


In [12]:
import numpy as np
import pandas as pd
import scipy.signal as signal
from scipy.signal import butter, filtfilt, lfilter

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Preprocessing functions
def notch_filter(data, fs, freq=50.0, Q=30.0):
    """
    Apply a notch filter to remove a specific frequency (e.g., 50 Hz).
    """
    b, a = signal.iirnotch(freq / (fs / 2), Q)
    return signal.filtfilt(b, a, data)

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    # y = lfilter(b, a, data)
    y = filtfilt(b, a, data)
    return y

In [13]:
# check how many rows have size > 256
print(df_emotiv[df_emotiv['size'] > 256].shape[0])
print(df_emotiv[df_emotiv['size'] == 256].shape[0])

731192
173614


In [14]:
filtered_df = df_emotiv[df_emotiv['size'] > 256]
filtered_df['data'] = filtered_df['data'].apply(lambda x: x[:256])

/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_10117/2735685688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['data'] = filtered_df['data'].apply(lambda x: x[:256])


In [18]:
# Apply notch filter
filtered_df['notched_data'] = filtered_df['data'].apply(lambda x: notch_filter(x, fs=128, freq=50.0, Q=30.0))

# Apply bandpass filter
filtered_df['filtered_data'] = filtered_df['notched_data'].apply(lambda x: bandpass_filter(x, fs=128, lowcut=0.5, highcut=50.0, order=4))


/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_10117/1472132230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['notched_data'] = filtered_df['data'].apply(lambda x: notch_filter(x, fs=128, freq=50.0, Q=30.0))
/var/folders/wy/50c5sv7n4wsfhhww_wwjz2tr0000gn/T/ipykernel_10117/1472132230.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['filtered_data'] = filtered_df['notched_data'].apply(lambda x: bandpass_filter(x, fs=128, lowcut=0.5, highcut=50.0, order=4))


In [20]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
from PIL import Image

def generate_concatenated_spectrogram(data, fs=128, nperseg=64):
    num_channels = data.shape[0]
    spectrogram_images = []
    
    for channel in range(num_channels):
        signal_data = data[channel, :]
        # Compute STFT
        f, t, Zxx = signal.stft(signal_data, fs=fs, nperseg=nperseg)
        # Take the magnitude of the STFT
        magnitude = np.abs(Zxx)
        # Convert to dB
        magnitude_db = 20 * np.log10(magnitude + 1e-8)
        # Normalize the spectrogram
        magnitude_db -= magnitude_db.min()
        magnitude_db /= magnitude_db.max()
        # Apply a colormap to convert to RGB
        colormap = plt.get_cmap('jet')
        spectrogram_colored = colormap(magnitude_db)
        # Convert to 8-bit unsigned integers
        spectrogram_uint8 = (spectrogram_colored[:, :, :3] * 255).astype(np.uint8)
        spectrogram_images.append(spectrogram_uint8)
    
    # Concatenate images horizontally
    concatenated_image = np.hstack(spectrogram_images)
    # Resize the image to 64x64 pixels
    final_image = Image.fromarray(concatenated_image).resize((64, 64), resample=Image.BICUBIC)
    # Convert back to NumPy array
    final_array = np.array(final_image)
    return final_array


In [ ]:
channels = np.unique(filtered_df['channel'])

grouped_df = filtered_df.groupby("event")

df_final = pd.DataFrame(columns=filtered_df.columns)

for event, group in grouped_df:
    # initialize a new row
    channel_vals = []
    for channel in channels:
        channel_data = group[group["channel"] == channel]["filtered_data"].values[0]
        channel_vals.append(channel_data)
    new_row = {
        "id": group["id"].values[0],  # Use the first ID for the group
        "event": event,               # Use the current event
        "device": group["device"].values[0],  # Use the first device for the group
        "channel": "all",             # Consolidate channels
        "code": group["code"].values[0],  # Use the first code
        "size": group["size"].values[0],  # Use the first size
        "data": np.concatenate(group["data"].values),  # Concatenate data arrays
        "stft": np.concatenate(group["stft"].values, axis=0),  # Concatenate along frequency/time axis
        "spectogram": generate_concatenated_spectrogram(np.array(channel_vals), fs=128, nperseg=64)
    }
    df_final = pd.concat([df_final, pd.DataFrame([new_row])])

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
X_train = []

for i in 